# Pipelines de treinamento: KNN

Esse notebook contém as pipelines de treinamento usadas na obtenção do melhor modelo classificador do problema desenvolvido no EP2.

O modelo KNN aceita diversas parametrizações diferentes - veja documentação: https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html#sklearn.neighbors.KNeighborsClassifier - aqui exploramos três: *neighbors* e *weights*.

As **features** que exploramos aqui são:  

1. Bag of Words  
2. TF/TF-IDF  
3. CHAR NGrams  
4. Embeddings

Os **modelos de embeddings** que exploramos aqui são:  

1. BAAI bge-3
2. Google Gemma 300m 

PS: Os embeddings Gemma 300m eu ainda NÃO CONSEGUI GERAR. Então, não precisa testar a parte que roda com eles no EP.

Você pode encontrar uma execução já parametrizada do melhor modelo encontrado por essas pipelines no notebook model.ipynb, **que é nossa versão de entrega do EP2**.

## Bootstrap Imports

In [ ]:
import pandas as pd
import numpy as np

import os
import sys
from pathlib import Path

### Teste de importação: Lib.utils do projeto

In [ ]:
filedir = Path(os.getcwd())
base_path = filedir.resolve().parents[3]
sys.path.append(str(base_path))

from Lib.utils import printhello
printhello()

HELLO!


## Configura variáveis de execução

In [ ]:
sep = ";"
dec = ","
quotech = "\""
encoding = "latin-1"


EP_dir = "EP2"
CSV_input_name = "ep2-train.csv"
path_to_archive = f"../../../../Traindata/{EP_dir}/{CSV_input_name}"


do_print = True
if do_print:
    print(f"Path to csv input is:  {path_to_archive}")

Path to csv input is:  ../../../../Traindata/EP2/ep2-train.csv


### Configure variáveis de reprodutibilidade

In [ ]:
random_state = 12345

### Lista de melhores modelos

In [ ]:
best_models_list = []

## Pré-tratamento de dados

### Importar dados do csv

In [ ]:
df = pd.read_csv(path_to_archive, na_values=['na'],
sep=sep,
decimal=dec,
quotechar=quotech,
encoding=encoding,
encoding_errors='strict')
print(df.shape)
print(df.columns)

(43678, 2)
Index(['req_text', 'profession'], dtype='object')


### Embaralhamento dos dados

O .csv de entrada tem alto ordenamento dos inputs por classe. Carregá-los dessa maneira nos modelos p/ treinamento introduz viés, então é preciso embaralhar os dados para garantir randomicidade. 
As classes em sklearn.model_selection - como a StratfiedKFold usada mais a frente - implementam parâmetro shuffle="", que pode ser passado como True para embaralhar mais os dados.

Note que é importante também garantir a reprodutibilidade do embaralhamento, especificando um valor hardcoded (Neste caso random_state=100)

In [ ]:
print("Shape antes do shuffle:", df.shape)

df = df.sample(frac=1, random_state=random_state).reset_index(drop=True) #NAO MUDE random_state, essa variavel DEVE valer 12345, ou QUEBRARÁ REPRODUTIBILIDADE dos experimentos

print("Shape depois do shuffle:", df.shape)

Shape antes do shuffle: (43678, 2)
Shape depois do shuffle: (43678, 2)


### Limpeza dos dados

In [ ]:
from Lib.utils import clean_text
#def clean_text(text, do_lowercase: bool, rem_emails: bool, rem_urls: bool, normalize_whitespaces: bool):

df['req_text_cleaned'] = df['req_text'].apply(lambda row_text: clean_text(
        row_text, 
        do_lowercase=True, 
        rem_emails=True, 
        rem_urls=True, 
        normalize_whitespaces=True
    ))

df['req_text'] = df['req_text_cleaned']
df = df.drop(columns=['req_text_cleaned']) # Remove a coluna temporária

# Treino dos modelos KNN 

## Imports

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectKBest, chi2
from imblearn.under_sampling import RandomUnderSampler
from sklearn.preprocessing import Normalizer
from sklearn.decomposition import PCA

### Disable Warnings

In [ ]:
import warnings
from sklearn.exceptions import ConvergenceWarning

# Ignora warnings de ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)

# Ignora UserWarnings específicos de l1_ratio etc
warnings.filterwarnings("ignore", category=UserWarning)

## Training Features: Embeddings

### Algoritmo & parâmetros

In [ ]:
knn = KNeighborsClassifier()

knn_parameters = {
    'n_neighbors': range(2, 40),
    'weights': ['uniform', 'distance']
}

# Treino c/ embeddings do modelo: BAAI-bge-3

## Importação dos embeddings

In [ ]:
X_baai = np.load('../../Embeddings/npys/gen_baai_bge_3.ipynb')
y_baai = df["profession"].values

Gerando BAAI embeddings
Shape dos embeddings (X): (500, 1024)
embeddings gerados


## Undersampling e Normalização

In [ ]:
X_baai = np.load('../../Embeddings/npys/gen_baai_bge_3.ipynb')
y_baai = df["profession"].values

undersampler = RandomUnderSampler(
    sampling_strategy={
        'classeGov': 10303,
        'classeAcad': 10303,
        'classePriv': 10303,
    },
    random_state=random_state
)
normalizer = Normalizer(norm='l2') 

X_baai_under, y_baai_under = undersampler.fit_resample(X_baai, y_baai) #Gera X e y undersampleados

In [ ]:
X_baai_norm_under = normalizer.fit_transform(X_baai_under) #Nomaliza vetores de treino undersampleados
X_baai_norm = normalizer.fit_transform(X_baai) #Nomaliza vetores de treino originais

## Aplicação dos PCAs & Treino dos modelos

### PCA 10

#### Full sample

In [ ]:
X_baai_pca_10 = PCA(n_components=10, random_state=random_state).fit_transform(X_baai_norm)

knn_grid_baai_10 = GridSearchCV(knn, knn_parameters, 
                                       cv=10, n_jobs=-1, scoring="accuracy", verbose=1, error_score=np.nan)
# 3. Fitar (Treinar) usando os embeddings pré-calculados
knn_grid_baai_10.fit(X_baai_pca_10, y_baai)

In [ ]:
print("Melhor acurácia média:", knn_grid_baai_10.best_score_)
print("Melhores parâmetros:", knn_grid_baai_10.best_params_)

best_models_list.append({
    "features": "embeddings baai-bge-3",
    "PCA": "10",
    "undersampled": False,
    "accuracy": knn_grid_baai_10.best_score_,
    "params": knn_grid_baai_10.best_params_.copy(),
})

#### Undersample

In [ ]:
X_baai_pca_10_under = PCA(n_components=10, random_state=random_state).fit_transform(X_baai_norm_under)

knn_grid_baai_10_under = GridSearchCV(knn, knn_parameters, 
                                       cv=10, n_jobs=-1, scoring="accuracy", verbose=1, error_score=np.nan)
# 3. Fitar (Treinar) usando os embeddings pré-calculados
knn_grid_baai_10_under.fit(X_baai_pca_10_under, y_baai)

In [ ]:
print("Melhor acurácia média:", knn_grid_baai_10_under.best_score_)
print("Melhores parâmetros:", knn_grid_baai_10_under.best_params_)

best_models_list.append({
    "features": "embeddings baai-bge-3",
    "PCA": "10",
    "undersampled": True,
    "accuracy": knn_grid_baai_10_under.best_score_,
    "params": knn_grid_baai_10_under.best_params_.copy(),
})

#### Liberação de Memória

In [ ]:
del X_baai_pca_10
del knn_grid_baai_10

del X_baai_pca_10_under
del knn_grid_baai_10_under

gc.collect()

## Treino c/ embeddings do modelo: Google-Gemma-300m

#### Importação dos embeddings

In [ ]:
# 1. Gerar os dados X e Y
X_gemma = np.load('../../Embeddings/npys/gen_google_gemma_300m.ipynb')
y_gemma = df["profession"].values

#### Treino do modelo 

In [ ]:
embed_grid_gemma = GridSearchCV(embed_pipeline, embed_parameters, 
                                       cv=10, n_jobs=-1, scoring="accuracy", verbose=1, error_score=np.nan)
# 3. Fitar (Treinar) usando os embeddings pré-calculados
embed_grid_gemma.fit(X_gemma, y_gemma)

In [ ]:
print("Melhor acurácia média:", embed_grid_gemma.best_score_)
print("Melhores parâmetros:", embed_grid_gemma.best_params_)

best_models_list.append({
    "features": "embeddings google-gemma-300m",
    "accuracy": embed_grid_gemma.best_score_,
    "params": embed_grid_gemma.best_params_,
})

# Seleciona melhores parâmetros

In [ ]:
best_score = -1
best = 0
for idx, candidate in enumerate(best_models_list):
    if candidate["accuracy"] > best_score:
        best = idx
        best_score = candidate["accuracy"]

print(f"O melhor classificador encontrado pelas pipelines é -->    feature={best_models_list[best]["features"]}\n")
print(f"Melhor acucácia encontrada:  {best_models_list[best]['accuracy']}")
print(f"Melhores parametros encontrados:  {best_models_list[best]['params']}")